## **ПРАКТИЧЕСКОЕ ЗАДАНИЕ**

**Задание**

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов.

In [26]:
# импортируем библиотеки

import tensorflow as tf
import numpy as np
import os
import time

In [27]:
# загрузим текст Евгения Онегина

path_to_file = '/content/drive/MyDrive/Colab Notebooks/NLP/evgenyi_onegin.txt'
# path_to_file = '/content/drive/MyDrive/Копия evgenyi_onegin.txt'

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text (number of characters in it): {} characters'.format(len(text)))

Length of text (number of characters in it): 286984 characters


In [28]:
print(text[:1000])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высоких дум и простоты;
                        Но так и быть - рукой пристрастной
                        Прими собранье пестрых глав,
                        Полусмешных, полупечальных,
                        Простонародных, идеальных,
                        Небрежный плод моих забав,
                        Бессонниц, легких вдохновений,
                        Незрелых и увядших лет,
                        Ума холодных наблюдений
                        И сердца горестных замет.

            

In [29]:
# Посмотрим на количество уникальных символов в тексте
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [30]:
# Сделаем словарь из уникальных символов и индексов 

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [31]:
text_as_int[:500], text[:500], len(text_as_int), len(text)

(array([ 71, 110, 104, 109, 116,  99, 112, 103, 115,   1,  87, 104, 115,
        102, 104, 104, 101, 107, 122,   1,  85, 118, 123, 109, 107, 112,
          0,   0,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,  76, 101, 102, 104, 112,
        107, 108,   1,  84, 112, 104, 102, 107, 112,   0,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,  86, 113, 111,  99, 112,   1, 101,   1, 116, 117,
        107, 120,  99, 120,   0,   0,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,  83, 104,   1, 111, 126, 116, 110, 130,   1,
        102, 113, 115, 103, 126, 108,   1, 116, 101, 104, 117,   1, 106,
         99, 100,  99, 101, 107, 117, 127,   7,   0

In [32]:
# Переведем текст в тензор

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [33]:
# Разделим данные на батчи

seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [34]:
# Функция разделения текста на данные для обучения и таргет

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [35]:
# Разделим данные. Происходит смещение не 1 символ

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [36]:
# Зададим некоторые параметры, перемешаем данные и поделим на батчи

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [37]:
# Зададим архитектуру модели генерации текста

def build_model(vocab_size, embedding_dim, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        # tf.keras.layers.GRU(1024,
        #                     return_sequences=True,
        #                     stateful=False,
        #                     recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
         tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [38]:
# Построим модель

EMBEDDING_DIM = 256
model = build_model(vocab_size=len(vocab),
                    embedding_dim=EMBEDDING_DIM,
                    batch_size=BATCH_SIZE)

In [39]:
# Посмотрим на выходную размерность предсказаний

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           33536     
                                                                 
 lstm_8 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 lstm_9 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_10 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_11 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 dense_2 (Dense)             (64, None, 131)           134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899


In [41]:
# example_batch_predictions[0]

In [42]:
# Посмотрим на пример генерации необученной модели

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 'ша его\n                        Не слышит горя моего.\n\n                                     XXXI\n\n   '

Next Char Predictions: 
 '\'DqQэ;h(IсШГЖ)oFБь"ШЗ1КМАXОeк,H}MlzkVэVШ8я1NfА4шLAзАуПdЛФ8йeЖ1юdз2шfдцТzжkGwIyЛъtп09X0АEWlЯLЖh\nRААuB'


In [43]:
# Зададим функцию потерь

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8770537


In [44]:
# место для хранения checkpoint
checkpoint_dir = './training_checkpoints'
# Имя файла checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=44*100, # сохраняем каждую сотую эпоху - в эпохе 44 батча (это видно при запуске обучения)
    save_weights_only=True,
    )

In [45]:
# Обучим модель

EPOCHS = 100
model.compile(optimizer='adam', loss=loss, metrics='accuracy')
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
44/44 [==============================] - 18s 312ms/step - loss: 2.3679 - accuracy: 0.5487
Epoch 2/100
44/44 [==============================] - 14s 316ms/step - loss: 1.7852 - accuracy: 0.5647
Epoch 3/100
44/44 [==============================] - 14s 319ms/step - loss: 1.6271 - accuracy: 0.5804
Epoch 4/100
44/44 [==============================] - 14s 321ms/step - loss: 1.4885 - accuracy: 0.5956
Epoch 5/100
44/44 [==============================] - 15s 324ms/step - loss: 1.4074 - accuracy: 0.6046
Epoch 6/100
44/44 [==============================] - 15s 326ms/step - loss: 1.3611 - accuracy: 0.6132
Epoch 7/100
44/44 [==============================] - 15s 326ms/step - loss: 1.3337 - accuracy: 0.6207
Epoch 8/100
44/44 [==============================] - 15s 326ms/step - loss: 1.2943 - accuracy: 0.6291
Epoch 9/100
44/44 [==============================] - 15s 325ms/step - loss: 1.2878 - accuracy: 0.6335
Epoch 10/100
44/44 [==============================] - 15s 328ms/step - loss: 1.245

In [46]:
#Находим имя файла последней сохраненной контрольной точки
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [47]:
#строим модель
model = build_model(len(vocab), 256, batch_size=1)

#загружаем веса из последней сохраненной контрольной точки в модель
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# model.build(tf.TensorShape([1, None]))

In [48]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 256)            33536     
                                                                 
 lstm_12 (LSTM)              (1, None, 1024)           5246976   
                                                                 
 lstm_13 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_14 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_15 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899


In [49]:
# Посмотрим на результаты генерации текста обученной модели

def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже не взвений
                           В тебе не встретишь, свет пустой.

                                  Скоркаться не дожил,
                     XVI

                          Уж темно: в санки он садится.
                                    Врова полна, дде он Жремен страшный вред,
                           Читать в глаза своим любезным
                          Сосед наш неуч; сумасбит.
                          Что пылких душ неосторожность
                        Самолюбивую ничтожнят".
                         Сердечный друг, желанный друг,
                         Почетный гражданин кулодом {36}.
                        Пошли же внука своего.
                         "Мой друг, вот бог тебе порука".
       И он, все клуши степной
                    Об,
                         Татьяну милую мою!

                                     IV

                      Так мысль ее далече бродит:
                           И начитаться не могу.
                        Ц